In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import math
import re
import html
import os.path
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
file_location = './Stocktwit Raw.csv'

In [3]:
st_raw = pd.read_csv(file_location, header=0, encoding = 'unicode_escape')

In [4]:
st_raw.head()

,Title,Title_link,st_2jbUWIi,st_3SL2gug,st_1NNeqUz,st_28bQfzV,st_1NNeqUz1,st_3xRimaf,lib_XwnOHoV,likes
0,jekoz,https://stocktwits.com/jekoz,https://s3.amazonaws.com/st-avatars/images/def...,$DIS nothing can get this over 140. Maybe $AAP...,$DIS,"Sep 17th, 8:40 am",$AAPL,NaN,NaN,NaN
1,sleekOptions,https://stocktwits.com/sleekOptions,https://avatars.stocktwits.com/production/1442...,$AAPL [Aug-30 210 Calls] up +30.43 % Alerted ...,$AAPL,"Sep 17th, 8:28 am",NaN,NaN,NaN,NaN
2,sleekOptions,https://stocktwits.com/sleekOptions,https://avatars.stocktwits.com/production/1442...,Peak profit for the last 6 expired option aler...,$AAPL,"Sep 17th, 8:26 am",NaN,NaN,NaN,NaN
3,sleekOptions,https://stocktwits.com/sleekOptions,https://avatars.stocktwits.com/production/1442...,"$AAPL Last six months, 41 option alerts peaked...",$AAPL,"Sep 17th, 8:25 am",NaN,NaN,NaN,NaN
4,klrkawa,https://stocktwits.com/klrkawa,https://avatars.stocktwits.com/production/1520...,$AAPL $ROKU $DE $HD $TGT down. Not looking go...,$AAPL,"Sep 17th, 8:11 am",$ROKU,1.0,Bearish,1.0


In [4]:
st = st_raw.iloc[:,3:6]
st.drop(['st_1NNeqUz'], inplace=True, axis=1)

In [5]:
st.head(30)

,st_3SL2gug,st_28bQfzV
0,$DIS nothing can get this over 140. Maybe $AAP...,"Sep 17th, 8:40 am"
1,$AAPL [Aug-30 210 Calls] up +30.43 % Alerted ...,"Sep 17th, 8:28 am"
2,Peak profit for the last 6 expired option aler...,"Sep 17th, 8:26 am"
3,"$AAPL Last six months, 41 option alerts peaked...","Sep 17th, 8:25 am"
4,$AAPL $ROKU $DE $HD $TGT down. Not looking go...,"Sep 17th, 8:11 am"
5,$AMZN $GOOGL $AAPL $VOOG always a solid buy fo...,"Sep 17th, 8:02 am"
6,$AAPL China says 'vice ministerial' officials ...,"Sep 17th, 7:58 am"
7,$AAPL $NET $SNAP $OXY it's awesome,"Sep 17th, 7:23 am"
8,$AAPL everyone please report @ceobrattwatking....,"Sep 17th, 7:19 am"
9,$AAPL ??,"Sep 17th, 7:10 am"


In [6]:
st.columns=['message', 'datetime']

In [7]:
st

,message,datetime
0,$DIS nothing can get this over 140. Maybe $AAP...,"Sep 17th, 8:40 am"
1,$AAPL [Aug-30 210 Calls] up +30.43 % Alerted ...,"Sep 17th, 8:28 am"
2,Peak profit for the last 6 expired option aler...,"Sep 17th, 8:26 am"
3,"$AAPL Last six months, 41 option alerts peaked...","Sep 17th, 8:25 am"
4,$AAPL $ROKU $DE $HD $TGT down. Not looking go...,"Sep 17th, 8:11 am"
5,$AMZN $GOOGL $AAPL $VOOG always a solid buy fo...,"Sep 17th, 8:02 am"
6,$AAPL China says 'vice ministerial' officials ...,"Sep 17th, 7:58 am"
7,$AAPL $NET $SNAP $OXY it's awesome,"Sep 17th, 7:23 am"
8,$AAPL everyone please report @ceobrattwatking....,"Sep 17th, 7:19 am"
9,$AAPL ??,"Sep 17th, 7:10 am"


In [8]:
def pr(a):
    print(st["message"][a])

In [62]:
st["message"][x] = st['message'].apply(lambda x: ' '.join(re.findall('[A-Z][^A-Z]*', x))

In [9]:
"""
    preprocess the data in file location and saves it as a csv file (appending '_preprocessed' before '.csv). The preprocessing us in following ways:
    1) extract message and datetime columns.
    2) sort according to datetime in descending order (newest first)
    3) remove links, @ and $ references, extra whitespaces, extra '.', digits, slashes, hyphens
    4) decode html entities
    5) convert everything to lower case
"""
st.sort_values(by='datetime', ascending=False)

st['message'] = st['message'].apply(lambda x: html.unescape(x))
st['message'] = st['message'].apply(lambda x: re.sub(r'(www\.|https?://).*?(\s|$)|@.*?(\s|$)|\$.*?(\s|$)|\d|\%|\\|/|-|_', ' ', x))
st['message'] = st['message'].apply(lambda x: re.sub(r'\.+', '. ', x))
st['message'] = st['message'].apply(lambda x: re.sub(r'\,+', ', ', x))
st['message'] = st['message'].apply(lambda x: re.sub(r'\?+', '? ', x))
st['message'] = st['message'].apply(lambda x: re.sub(r'\s+', ' ', x))
st['message'] = st['message'].apply(lambda x: x.lower())

st.to_csv(file_location[:-4]+'_preprocessed.csv', index=False)

In [13]:
st.sort_values(by='datetime', ascending=False)

st['message'] = st['message'].apply(lambda x: html.unescape(x))
st['message'] = st['message'].apply(lambda x: re.sub(r'(www\.|https?://).*?(\s|$)|@.*?(\s|$)|\$.*?(\s|$)|\d|\%|\\|/|-|_', ' ', x))
st['message'] = st['message'].apply(lambda x: re.sub(r'\s+', ' ', x))

st.to_csv(file_location[:-4]+'_preprocessed2.csv', index=False)

In [14]:
st2 = pd.read_csv('./Stocktwit Raw_preprocessed2.csv', header=0)

In [15]:
st2.head()

,message,datetime
0,nothing can get this over . Maybe buyout will...,"Sep 17th, 8:40 am"
1,[Aug Calls] up + . Alerted at on Aug : AM Pea...,"Sep 17th, 8:28 am"
2,Peak profit for the last expired option alerts...,"Sep 17th, 8:26 am"
3,"Last six months, option alerts peaked above p...","Sep 17th, 8:25 am"
4,down. Not looking good for Tuesday. Fed will ...,"Sep 17th, 8:11 am"


In [16]:
for x in range(10):
    print(st["message"][x])

 nothing can get this over . Maybe buyout will do it. Like tops i guess.
 [Aug Calls] up + . Alerted at on Aug : AM Peak after alert on 
Peak profit for the last expired option alerts for . | . | . | . | . | . |
 Last six months, option alerts peaked above percent after they were posted
 down. Not looking good for Tuesday. Fed will make it worse. Market Watch: Asian markets slip as tensions remain high after attack on Saudi oil facilities
 always a solid buy for portfolio. Also, now over followers & gaining, you guys are awesome! ??
 China says 'vice ministerial' officials will be in Washington for trade talks cnbc.com china s...
 it's awesome
 everyone please report All he does is spam every company with technical analysis Videos that arent even analyzing anything just a stupid crappy way to get you views for his pathetic YouTube channel
 ??


In [ ]:
"""
    extract keywords from labelled stocktwits data for improved accuracy in scoring for each labelled message do
    1) tokenize the message
    2) perform POS tagging
    3) if a sense is present in wordnet then, lemmatize the word and remove stop words else ignore the word
    remove intersections from the two lists before saving
"""

        st_pre = LoadData.get_labelled_data()
        bullish_keywords = set()
        bearish_keywords = set()
        lemmatizer = WordNetLemmatizer()
        stop_words = set(stopwords.words('english'))
        for index, row in dataFrame.iterrows():
            tokens = word_tokenize(row['message'])
            pos = pos_tag(tokens)
            selected_tags = set()

            for i in range(len(pos)):
                if len(wordnet.synsets(pos[i][0])):
                    if pos[i][1].startswith('J'):
                        selected_tags.add(lemmatizer.lemmatize(pos[i][0], 'a'))
                    elif pos[i][1].startswith('V'):
                        selected_tags.add(lemmatizer.lemmatize(pos[i][0], 'v'))
                    elif pos[i][1].startswith('N'):
                        selected_tags.add(lemmatizer.lemmatize(pos[i][0], 'n'))
                    elif pos[i][1].startswith('R'):
                        selected_tags.add(lemmatizer.lemmatize(pos[i][0], 'r'))
            selected_tags -= stop_words

            if row['sentiment'] == 'Bullish':
                bullish_keywords = bullish_keywords.union(selected_tags)
            elif row['sentiment'] == 'Bearish':
                bearish_keywords = bearish_keywords.union(selected_tags)

        updated_bullish_keywords = bullish_keywords - bearish_keywords
        updated_bearish_keywords = bearish_keywords - bullish_keywords
        with open('data-extractor/lexicon_bullish_words.txt', 'a') as file:
            for word in updated_bullish_keywords:
                file.write(word+"\n")
        with open('data-extractor/lexicon_bearish_words.txt', 'a') as file:
            for word in updated_bearish_keywords:
                file.write(word+"\n")

In [2]:
"""
    handle preprocessing and loading of data.
"""

import html
import os.path
import pandas as pd
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem.wordnet import WordNetLemmatizer

class LoadData:

    @classmethod
    def preprocess_stocktwits_data(cls, file_location, columns=['datetime', 'message']):
        """
            preprocess the data in file location and saves it as a csv file (appending
            '_preprocessed' before '.csv). The preprocessing us in following ways:
            1) extract message and datetime columns.
            2) sort according to datetime in descending order (newest first)
            3) remove links, @ and $ references, extra whitespaces, extra '.', digits, slashes,
            hyphons
            4) decode html entities
            5) convert everything to lower case
        """

        if 'datetime' in columns:
            dataFrame = pd.read_csv(file_location, usecols=columns, parse_dates=['datetime'], infer_datetime_format=True)
            dataFrame.sort_values(by='datetime', ascending=False)
        else:
            dataFrame = pd.read_csv(file_location, usecols=columns)

        dataFrame['message'] = dataFrame['message'].apply(lambda x: html.unescape(x))
        dataFrame['message'] = dataFrame['message'].apply(lambda x: re.sub(r'(www\.|https?://).*?(\s|$)|@.*?(\s|$)|\$.*?(\s|$)|\d|\%|\\|/|-|_', ' ', x))
        dataFrame['message'] = dataFrame['message'].apply(lambda x: re.sub(r'\.+', '. ', x))
        dataFrame['message'] = dataFrame['message'].apply(lambda x: re.sub(r'\,+', ', ', x))
        dataFrame['message'] = dataFrame['message'].apply(lambda x: re.sub(r'\?+', '? ', x))
        dataFrame['message'] = dataFrame['message'].apply(lambda x: re.sub(r'\s+', ' ', x))
        dataFrame['message'] = dataFrame['message'].apply(lambda x: x.lower())

        dataFrame.to_csv(file_location[:-4]+'_preprocessed.csv', index=False)

    @classmethod
    def labelled_data_lexicon_analysis(cls):
        """
            extract keywords from labelled stocktwits data for improved accuracy in scoring
            for each labelled message do
            1) tokenize the message
            2) perform POS tagging
            3) if a sense is present in wordnet then, lemmatize the word and remove stop words else ignore the word
            remove intersections from the two lists before saving
        """

        dataFrame = LoadData.get_labelled_data()
        bullish_keywords = set()
        bearish_keywords = set()
        lemmatizer = WordNetLemmatizer()
        stop_words = set(stopwords.words('english'))
        for index, row in dataFrame.iterrows():
            tokens = word_tokenize(row['message'])
            pos = pos_tag(tokens)
            selected_tags = set()

            for i in range(len(pos)):
                if len(wordnet.synsets(pos[i][0])):
                    if pos[i][1].startswith('J'):
                        selected_tags.add(lemmatizer.lemmatize(pos[i][0], 'a'))
                    elif pos[i][1].startswith('V'):
                        selected_tags.add(lemmatizer.lemmatize(pos[i][0], 'v'))
                    elif pos[i][1].startswith('N'):
                        selected_tags.add(lemmatizer.lemmatize(pos[i][0], 'n'))
                    elif pos[i][1].startswith('R'):
                        selected_tags.add(lemmatizer.lemmatize(pos[i][0], 'r'))
            selected_tags -= stop_words

            if row['sentiment'] == 'Bullish':
                bullish_keywords = bullish_keywords.union(selected_tags)
            elif row['sentiment'] == 'Bearish':
                bearish_keywords = bearish_keywords.union(selected_tags)

        updated_bullish_keywords = bullish_keywords - bearish_keywords
        updated_bearish_keywords = bearish_keywords - bullish_keywords
        with open('data-extractor/lexicon_bullish_words.txt', 'a') as file:
            for word in updated_bullish_keywords:
                file.write(word+"\n")
        with open('data-extractor/lexicon_bearish_words.txt', 'a') as file:
            for word in updated_bearish_keywords:
                file.write(word+"\n")

    @classmethod
    def get_stocktwits_data(cls, symbol):
        """
            get_data loads the preprocessed data of 'symbol' from data-extractor
            and returns a pandas dataframe with columns [message(object), datetime(datetime64[ns])].
        """

        file_location = 'data-extractor/stocktwits_'+symbol+'_preprocessed.csv'
        if os.path.isfile(file_location) is False:
            LoadData.preprocess_stocktwits_data('data-extractor/stocktwits_'+symbol+'.csv')

        dataFrame = pd.read_csv(file_location)
        return dataFrame

    @classmethod
    def get_price_data(cls, symbol):
        """
            loads the price data of 'symbol' from data-extractor
            and returns a pandas dataframe with columns [Date(datetime64[ns]), Opening Price(float64), Closing Price(float64), Volume(float64)].
        """

        file_location = 'data-extractor/stock_prices_'+symbol+'.csv'
        dataFrame = pd.read_csv(file_location, usecols=['Date', 'Opening Price', 'Closing Price', 'Volume'], parse_dates=['Date'], infer_datetime_format=True)
        return dataFrame

    @classmethod
    def get_labelled_data(cls, type='complete'):
        """
            get_labelled_data loads the preprocessed labelled data of stocktwits from data-extractor
            and returns a pandas dataframe with columns [sentiment(object), message(object)].
        """

        if type == 'complete':
            file_location = 'data-extractor/labelled_data_complete_preprocessed.csv'
            if os.path.isfile(file_location) is False:
                LoadData.preprocess_stocktwits_data('data-extractor/labelled_data_complete.csv', columns=['sentiment', 'message'])
        elif type == 'training':
            file_location = 'data-extractor/labelled_data_training_preprocessed.csv'
            if os.path.isfile(file_location) is False:
                LoadData.get_training_data()
        elif type == 'test':
            file_location = 'data-extractor/labelled_data_test_preprocessed.csv'
            if os.path.isfile(file_location) is False:
                LoadData.preprocess_stocktwits_data('data-extractor/labelled_data_test.csv', columns=['sentiment', 'message'])

        dataFrame = pd.read_csv(file_location)
        return dataFrame

    @classmethod
    def get_custom_lexicon(cls):
        """
            get custom lexicon of bearish and bullish words respectively
        """

        file_location1 = 'data-extractor/lexicon_bearish_words.txt'
        file_location2 = 'data-extractor/lexicon_bullish_words.txt'
        if os.path.isfile(file_location1) is False or os.path.isfile(file_location2) is False:
            LoadData.labelled_data_lexicon_analysis()

        dataFrameBearish = pd.read_csv(file_location1, header=None, names=['word'])
        dataFrameBullish = pd.read_csv(file_location2, header=None, names=['word'])
        return dataFrameBearish, dataFrameBullish

    @classmethod
    def get_training_data(cls):
        """
            get labelled training data with equal bearish and bullish messages
        """
        try:
            os.remove('data-extractor/labelled_data_training.csv')
        except OSError:
            pass

        dataFrame = LoadData.get_labelled_data(type='complete')
        dataFrameBearish = dataFrame[dataFrame['sentiment']=='Bearish']
        dataFrameBullish = dataFrame[dataFrame['sentiment']=='Bullish']
        dataFrameBearishTraining = dataFrameBearish
        dataFrameBullishTraining = dataFrameBullish[:len(dataFrameBearish)]

        dataFrameTraining = dataFrameBearishTraining.append(dataFrameBullishTraining, ignore_index=True).sample(frac=1).reset_index(drop=True)
        dataFrameTraining.to_csv('data-extractor/labelled_data_training_preprocessed.csv', index=False)

    @classmethod
    def combine_price_and_sentiment(cls, sentimentFrame, priceFrame):
        from datetime import timedelta

        """
            receive sentimentFrame as (date, sentiment, message) indexed by date and sentiment
            and priceFrame as (Date, Opening Price, Closing Price, Volume) and return a combined
            frame as (sentiment_calculated_bullish, sentiment_calculated_bearish,
            sentiment_actual_previous, tweet_volume_change, cash_volume, label)
        """

        dataFrame = pd.DataFrame()
        for date, df in sentimentFrame.groupby(level=0, sort=False):

            price_current = priceFrame[priceFrame['Date'] == date]
            if price_current.empty or date-timedelta(days=1) not in sentimentFrame.index:
                continue
            tweet_minus1 = sentimentFrame.loc[date-timedelta(days=1)]
            days = 1
            price_plus1 = priceFrame[priceFrame['Date'] == date+timedelta(days=days)]
            while price_plus1.empty:
                days += 1
                price_plus1 = priceFrame[priceFrame['Date'] == date+timedelta(days=days)]
            days = 1
            price_minus1 = priceFrame[priceFrame['Date'] == date-timedelta(days=days)]
            while price_minus1.empty:
                days += 1
                price_minus1 = priceFrame[priceFrame['Date'] == date-timedelta(days=days)]

            new_row = {}
            new_row['date'] = date
            new_row['sentiment_calculated_bullish'] = df.loc[(date, 'Bullish')]['message']
            new_row['sentiment_calculated_bearish'] = df.loc[(date, 'Bearish')]['message']
            new_row['sentiment_actual_previous'] = 1 if ((price_minus1.iloc[0]['Closing Price'] - price_minus1.iloc[0]['Opening Price']) >= 0) else -1
            new_row['tweet_volume_change'] = df['message'].sum() - tweet_minus1['message'].sum()
            new_row['cash_volume'] = price_current['Volume'].iloc[0]
            new_row['label'] = 1 if ((price_plus1.iloc[0]['Closing Price'] - price_current.iloc[0]['Closing Price']) >= 0) else -1
            print(new_row)
            dataFrame = dataFrame.append(new_row, ignore_index=True)

        return dataFrame

    @classmethod
    def aggregate_stock_price_data(cls):
        """
            compile stocktwits data for stock prediction analysis in the following form
            (date, sentiment_calculated_bullish, sentiment_calculated_bearish, sentiment_actual_previous, tweet_volume_change, cash_volume, label)
            we have choice to take previous n days sentiment_calculated and using label of next nth day
            returns dataframes for AAPL, AMZN, GOOGL respectively
        """

        if not (os.path.isfile('data-extractor/stocktwits_AAPL_sharedata.csv') and os.path.isfile('data-extractor/stocktwits_AMZN_sharedata.csv') and os.path.isfile('data-extractor/stocktwits_GOOGL_sharedata.csv')):

            from sklearn.externals import joblib
            file_location = 'naive_bayes_classifier.pkl'

            priceAAPL = LoadData.get_price_data('AAPL')
            priceAMZN = LoadData.get_price_data('AMZN')
            priceGOOGL = LoadData.get_price_data('GOOGL')

            sentimented_file = 'data-extractor/stocktwits_AAPL_withsentiment.csv'
            if os.path.isfile(sentimented_file) is False:
                tweet_classifier = joblib.load(file_location)
                dataAAPL = LoadData.get_stocktwits_data('AAPL')
                dataAAPL['sentiment'] = dataAAPL['message'].apply(lambda x: tweet_classifier.predict([x])[0])
                dataAAPL['datetime'] = dataAAPL['datetime'].apply(lambda x: x.date())
                dataAAPL.rename(columns={'datetime':'date'}, inplace=True)
                dataAAPL.to_csv('data-extractor/stocktwits_AAPL_withsentiment.csv', index=False)
            sentimented_file = 'data-extractor/stocktwits_AMZN_withsentiment.csv'
            if os.path.isfile(sentimented_file) is False:
                tweet_classifier = joblib.load(file_location)
                dataAMZN = LoadData.get_stocktwits_data('AMZN')
                dataAMZN['sentiment'] = dataAMZN['message'].apply(lambda x: tweet_classifier.predict([x])[0])
                dataAMZN['datetime'] = dataAMZN['datetime'].apply(lambda x: x.date())
                dataAMZN.rename(columns={'datetime':'date'}, inplace=True)
                dataAMZN.to_csv('data-extractor/stocktwits_AMZN_withsentiment.csv', index=False)
            sentimented_file = 'data-extractor/stocktwits_GOOGL_withsentiment.csv'
            if os.path.isfile(sentimented_file) is False:
                tweet_classifier = joblib.load(file_location)
                dataGOOGL = LoadData.get_stocktwits_data('GOOGL')
                dataGOOGL['sentiment'] = dataGOOGL['message'].apply(lambda x: tweet_classifier.predict([x])[0])
                dataGOOGL['datetime'] = dataGOOGL['datetime'].apply(lambda x: x.date())
                dataGOOGL.rename(columns={'datetime':'date'}, inplace=True)
                dataGOOGL.to_csv('data-extractor/stocktwits_GOOGL_withsentiment.csv', index=False)

            dataAAPL = pd.read_csv('data-extractor/stocktwits_AAPL_withsentiment.csv', parse_dates=['date'], infer_datetime_format=True)
            dataAMZN = pd.read_csv('data-extractor/stocktwits_AMZN_withsentiment.csv', parse_dates=['date'], infer_datetime_format=True)
            dataGOOGL = pd.read_csv('data-extractor/stocktwits_GOOGL_withsentiment.csv', parse_dates=['date'], infer_datetime_format=True)
            dataAAPL = dataAAPL.groupby(['date','sentiment'], sort=False).count()
            dataAMZN = dataAMZN.groupby(['date','sentiment'], sort=False).count()
            dataGOOGL = dataGOOGL.groupby(['date','sentiment'], sort=False).count()
            dataAAPL = LoadData.combine_price_and_sentiment(dataAAPL, priceAAPL)
            dataAMZN = LoadData.combine_price_and_sentiment(dataAMZN, priceAMZN)
            dataGOOGL = LoadData.combine_price_and_sentiment(dataGOOGL, priceGOOGL)
            dataAAPL.to_csv('data-extractor/stocktwits_AAPL_sharedata.csv', index=False)
            dataAMZN.to_csv('data-extractor/stocktwits_AMZN_sharedata.csv', index=False)
            dataGOOGL.to_csv('data-extractor/stocktwits_GOOGL_sharedata.csv', index=False)

        dataAAPL = pd.read_csv('data-extractor/stocktwits_AAPL_sharedata.csv', parse_dates=['date'], infer_datetime_format=True)
        dataAMZN = pd.read_csv('data-extractor/stocktwits_AMZN_sharedata.csv', parse_dates=['date'], infer_datetime_format=True)
        dataGOOGL = pd.read_csv('data-extractor/stocktwits_GOOGL_sharedata.csv', parse_dates=['date'], infer_datetime_format=True)

        return dataAAPL, dataAMZN, dataGOOGL

    @classmethod
    def get_stock_prediction_data(cls, symbol='ALL', type='training'):

        """
            get the training and test data for stock prediction in format
            (sentiment_calculated_bullish, sentiment_calculated_bearish, sentiment_actual_previous,
            tweet_volume_change, cash_volume, label)
            Standardize the data before using.
        """

        file_location = 'data-extractor/stockdata_'+symbol+'_'+type+'.csv'
        if not os.path.isfile(file_location):
            import numpy as np

            dataAAPL, dataAMZN, dataGOOGL = LoadData.aggregate_stock_price_data()
            combined_data = dataAAPL.append([dataAMZN, dataGOOGL], ignore_index=True)
            combined_data.sort_values('date')
            combined_data.drop(columns='date', inplace=True)
            combined_training, combined_test = np.split(combined_data.sample(frac=1), [int(.9*len(combined_data))])
            combined_training.to_csv('data-extractor/stockdata_ALL_training.csv', index=False)
            combined_test.to_csv('data-extractor/stockdata_ALL_test.csv', index=False)

            dataAAPL.sort_values('date')
            dataAAPL.drop(columns='date', inplace=True)
            AAPL_training, AAPL_test = np.split(dataAAPL.sample(frac=1), [int(.9*len(dataAAPL))])
            AAPL_training.to_csv('data-extractor/stockdata_AAPL_training.csv', index=False)
            AAPL_test.to_csv('data-extractor/stockdata_AAPL_test.csv', index=False)

            dataAMZN.sort_values('date')
            dataAMZN.drop(columns='date', inplace=True)
            AMZN_training, AMZN_test = np.split(dataAMZN.sample(frac=1), [int(.9*len(dataAMZN))])
            AMZN_training.to_csv('data-extractor/stockdata_AMZN_training.csv', index=False)
            AMZN_test.to_csv('data-extractor/stockdata_AMZN_test.csv', index=False)

            dataGOOGL.sort_values('date')
            dataGOOGL.drop(columns='date', inplace=True)
            GOOGL_training, GOOGL_test = np.split(dataGOOGL.sample(frac=1), [int(.9*len(dataGOOGL))])
            GOOGL_training.to_csv('data-extractor/stockdata_GOOGL_training.csv', index=False)
            GOOGL_test.to_csv('data-extractor/stockdata_GOOGL_test.csv', index=False)

        data = pd.read_csv(file_location)
        return data

In [3]:
"""
    perform sentiment analysis of stocktwits data
"""

import load_data
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from nltk import word_tokenize, pos_tag
from nltk.corpus import sentiwordnet as swn
from nltk.wsd import lesk
from nltk.stem.wordnet import WordNetLemmatizer

class SentimentAnalysis:

    @classmethod
    def get_sentiword_score(cls, message):
        """
            takes a message and performs following operations:
            1) tokenize
            2) POS tagging
            3) reduce text to nouns, verbs, adjectives, adverbs
            4) lemmatize the words
            for each selected tag, if more than one sense exists, performs word sense disambiguation
            using lesk algorithm and finally returns positivity score, negativity score from
            sentiwordnet lexicon
        """

        tokens = word_tokenize(message)
        pos = pos_tag(tokens)
        lemmatizer = WordNetLemmatizer()
        selected_tags = list()
        scores = list()

        for i in range(len(pos)):
            if pos[i][1].startswith('J'):
                selected_tags.append((lemmatizer.lemmatize(pos[i][0], 'a'), 'a'))
            elif pos[i][1].startswith('V'):
                selected_tags.append((lemmatizer.lemmatize(pos[i][0], 'v'), 'v'))
            elif pos[i][1].startswith('N'):
                selected_tags.append((lemmatizer.lemmatize(pos[i][0], 'n'), 'n'))
            elif pos[i][1].startswith('R'):
                selected_tags.append((lemmatizer.lemmatize(pos[i][0], 'r'), 'r'))

        # score list: [(sense name, pos score, neg score)]
        for i in range(len(selected_tags)):
            senses = list(swn.senti_synsets(selected_tags[i][0], selected_tags[i][1]))
            if len(senses) == 1:
                scores.append((senses[0].synset.name(), senses[0].pos_score(), senses[0].neg_score()))
            elif len(senses) > 1:
                sense = lesk(tokens, selected_tags[i][0], selected_tags[i][1])
                if sense is None:
                    # take average score of all original senses
                    pos_score = 0
                    neg_score = 0
                    for i in senses:
                        pos_score += i.pos_score()
                        neg_score += i.neg_score()
                    scores.append((senses[0].synset.name(), pos_score/len(senses), neg_score/len(senses)))
                else:
                    sense = swn.senti_synset(sense.name())
                    scores.append((sense.synset.name(), sense.pos_score(), sense.neg_score()))

        """
            there are a number of ways for aggregating sentiment scores
            1) sum up all scores
            2) average all scores (or only for non zero scores)
            3) (1) or (2) but only for adjectives
            4) if pos score greater than neg score +1 vote else -1 vote
            here we are summing up the positive and negative scores to be used by classifier.
            whenever we encounter a negative word, we reverse the positive and negative score.
        """

        # collected from word stat financial dictionary
        negation_words = list(open('data-extractor/lexicon_negation_words.txt').read().split())

        # final_score = 0
        # counter = 1
        # for score in scores:
        #     if any(score[0].startswith(x) for x in negation_words):
        #         counter *= -1
        #     else:
        #         if score[1] > score[2]:
        #             final_score += counter*score[1]
        #         elif score[1] < score[2]:
        #             final_score -= counter*score[2]

        counter = 1
        pos_score = 0
        neg_score = 0
        for score in scores:
            if any(score[0].startswith(x) for x in negation_words):
                counter *= -1
            else:
                if counter == 1:
                    pos_score += score[1]
                    neg_score += score[2]
                elif counter == -1:
                    pos_score += score[2]
                    neg_score += score[1]

        final_score = [pos_score, neg_score]
        print(final_score)
        return final_score

    @classmethod
    def sentiword_data_analysis(cls, symbol):
        file_location = 'data-extractor/stocktwits_'+symbol+'_sentiwordnet_scored.csv'
        if os.path.isfile(file_location) is False:
            dataFrame = load_data.LoadData.get_stocktwits_data(symbol)
            dataFrame['sentiwordnet_score'] = dataFrame.apply(lambda x: SentimentAnalysis.get_sentiword_score(x['message']), axis = 1)
            dataFrame.to_csv(file_location, index=False)

        dataFrame = pd.read_csv(file_location)
        plt.hist(dataFrame['sentiwordnet_score'], bins=np.arange(-3.5, 4, 0.1), label=symbol)
        plt.legend(loc='upper right')
        plt.show()

    @classmethod
    def labelled_data_sentiwordnet_analysis(cls):
        file_location = 'data-extractor/labelled_data_sentiwordnet_scored.csv'
        if os.path.isfile(file_location) is False:
            dataFrame = load_data.LoadData.get_labelled_data()
            dataFrame['sentiwordnet_score'] = dataFrame.apply(lambda x: SentimentAnalysis.get_sentiword_score(x['message']), axis = 1)
            dataFrame.to_csv(file_location, index=False)

        dataFrame = pd.read_csv(file_location)
        plt.hist(dataFrame[dataFrame['sentiment']=='Bullish']['sentiwordnet_score'], bins=np.arange(-3.5, 4, 0.1), label='Bullish', alpha=0.5)
        plt.hist(dataFrame[dataFrame['sentiment']=='Bearish']['sentiwordnet_score'], bins=np.arange(-3.5, 4, 0.1), label='Bearish', alpha=0.5)
        plt.legend(loc='upper right')
        plt.show()

ModuleNotFoundError: No module named 'load_data'